In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
from pathlib import Path
from models import EMGConvNet, TrainingManager, CrossValidationManager
from utils import save_experiment_log

In [2]:
# === Paths to data ===
X_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_X.npy")
y_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_y.npy")

# === Load Data ===
X = np.load(X_path)   # (5, 8, 230000)
y = np.load(y_path)   # (5, 51, 230000)

# === Extract windows by session ===
def extract_windows_by_session(X, y, window_size=500, stride=250):
    X_sessions, y_sessions = [], []
    sessions = X.shape[0]

    for sess in range(sessions):
        X_windows, y_targets = [], []
        for start in range(0, X.shape[2] - window_size + 1, stride):
            end = start + window_size
            x_window = X[sess, :, start:end]
            y_target = y[sess, :, end-1]
            X_windows.append(x_window)
            y_targets.append(y_target)
        X_sessions.append(np.stack(X_windows))
        y_sessions.append(np.stack(y_targets))

    return X_sessions, y_sessions

X_sessions, y_sessions = extract_windows_by_session(X, y)

# === Keep only first 4 sessions for CV ===
X_sessions_cv = X_sessions[:4]
y_sessions_cv = y_sessions[:4]

class EMGDataset(Dataset):
    def __init__(self, X, y, standardize=True):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

        if standardize:
            self.mean = self.X.mean(dim=(0, 2), keepdim=True)
            self.std = self.X.std(dim=(0, 2), keepdim=True)
            self.X = (self.X - self.mean) / (self.std + 1e-8)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [3]:
# EXAMPLE LOOK HERE :)


# Model and training configs
model_config = {
    "conv_layers_config": [(16, 5, 1), (32, 5, 2)],
    "fc_layers_config": [512, 256, 128, 64],
    "output_dim": 51,
    "verbose": False
}

training_config = {
    "lr": 1e-3,
    "epochs": 100,
    "batch_size": 64,
    "log_every": 1
}

# Cross-validation runner
cross_validator = CrossValidationManager(
    model_class=EMGConvNet,
    model_config=model_config,
    data=X_sessions_cv,
    labels=y_sessions_cv,
    training_config=training_config,
    dataset_class=EMGDataset,
    dataset_config={"standardize": True},
    n_folds=4
)

# Run and save
experiment_log = cross_validator.run()
save_experiment_log(experiment_log, path="logs/neuralnetwork_log.json")



===== Fold 1/4 =====
Epoch   1 | Train MSE: 261.0624 | Val MSE: 160.7208 | Val RMSE: 12.6020
Epoch   2 | Train MSE: 167.8161 | Val MSE: 180.6510 | Val RMSE: 13.3534
Epoch   3 | Train MSE: 154.7373 | Val MSE: 161.8548 | Val RMSE: 12.6668
Epoch   4 | Train MSE: 143.4857 | Val MSE: 143.5219 | Val RMSE: 11.9192
Epoch   5 | Train MSE: 132.8638 | Val MSE: 132.8527 | Val RMSE: 11.4198
Epoch   6 | Train MSE: 84.3853 | Val MSE: 110.1025 | Val RMSE: 10.3596
Epoch   7 | Train MSE: 77.9169 | Val MSE: 113.4969 | Val RMSE: 10.5457
Epoch   8 | Train MSE: 72.6037 | Val MSE: 74.1197 | Val RMSE: 8.5076
Epoch   9 | Train MSE: 61.8708 | Val MSE: 99.9431 | Val RMSE: 9.8297
Epoch  10 | Train MSE: 62.3921 | Val MSE: 73.2181 | Val RMSE: 8.4814
Epoch  11 | Train MSE: 59.4253 | Val MSE: 74.5697 | Val RMSE: 8.5140
Epoch  12 | Train MSE: 58.6777 | Val MSE: 67.7991 | Val RMSE: 8.1723
Epoch  13 | Train MSE: 63.2287 | Val MSE: 71.8354 | Val RMSE: 8.3808
Epoch  14 | Train MSE: 55.3891 | Val MSE: 67.4507 | Val RMSE: 

KeyboardInterrupt: 

In [4]:
import random
class ArchitectureSampler:
    def __init__(
        self,
        widths=(128, 256, 512, 768, 1024, 1536),
        depths=(2, 3, 4, 5),
        shapes=("increasing", "decreasing", "symmetric", "flat"),
        limit_per_shape=10,
        dropout_levels=(0.0, 0.05, 0.1),
        max_fc_depth=None,
        max_conv_depth=None,
        conv_templates=None,
        output_dim=51,
        seed=None,
        verbose=False
    ):
        self.widths = widths
        self.depths = depths
        self.shapes = shapes
        self.limit = limit_per_shape
        self.dropout_levels = dropout_levels
        self.max_fc_depth = max_fc_depth
        self.max_conv_depth = max_conv_depth
        self.output_dim = output_dim
        self.verbose = verbose
        self.conv_templates = conv_templates
        self.seed = seed

        if seed is not None:
            random.seed(seed)

        self._configs = []

    def _generate_shape(self, shape, depth):
        if shape == "increasing":
            return sorted(random.sample(self.widths, depth))
        elif shape == "decreasing":
            return sorted(random.sample(self.widths, depth), reverse=True)
        elif shape == "symmetric":
            half = sorted(random.sample(self.widths, depth // 2))
            return half + half[::-1] if depth % 2 == 0 else half + [random.choice(self.widths)] + half[::-1]
        elif shape == "flat":
            w = random.choice(self.widths)
            return [w] * depth
        else:
            raise ValueError(f"Unknown shape type: {shape}")

    def generate_fc_templates(self):
        fc_templates = {}
        for shape in self.shapes:
            for depth in self.depths:
                key = f"{shape}_d{depth}"
                fc_templates[key] = [
                    self._generate_shape(shape, depth)
                    for _ in range(self.limit)
                ]
        return fc_templates

    def generate_configs(self):
        fc_templates = self.generate_fc_templates()

        # fallback if no conv templates provided
        conv_templates = self.conv_templates or {
            "shallow_wide": [[(64, 5, 1)], [(128, 5, 1), (128, 3, 1)]],
            "deep_narrow": [[(32, 3, 1)] * 4, [(64, 3, 1)] * 5],
            "expanding": [[(32, 5, 1), (64, 3, 1), (128, 3, 1)]],
            "bottleneck": [[(128, 5, 1), (64, 3, 1), (32, 3, 1)]],
            "oscillating": [[(64, 5, 1), (128, 3, 1), (64, 3, 1)]],
        }

        configs = []

        for conv_name, conv_list in conv_templates.items():
            for fc_name, fc_list in fc_templates.items():
                for conv_cfg in conv_list:
                    if self.max_conv_depth and len(conv_cfg) > self.max_conv_depth:
                        continue
                    for fc_cfg in fc_list:
                        if self.max_fc_depth and len(fc_cfg) > self.max_fc_depth:
                            continue
                        for d in self.dropout_levels:
                            config = {
                                "conv_layers_config": list(conv_cfg),
                                "fc_layers_config": list(fc_cfg),
                                "conv_dropouts": [d] * len(conv_cfg),
                                "fc_dropouts": [d] * len(fc_cfg),
                                "output_dim": self.output_dim,
                                "verbose": self.verbose,
                                "conv_family": conv_name,
                                "fc_family": fc_name,
                                "dropout_level": d
                            }
                            configs.append(config)

        self._configs = configs
        return configs

    def get_all_configs(self):
        return self._configs

    def get_model_configs(self):
        keys = ["conv_layers_config", "fc_layers_config", "conv_dropouts", "fc_dropouts", "output_dim", "verbose"]
        return [{k: c[k] for k in keys} for c in self._configs]
    
    def shuffle_configs(self, seed=None):
        if seed is not None:
            random.seed(seed)
        random.shuffle(self._configs)


In [79]:
import uuid
import os
import traceback
from models import EMGConvNet  # delay import for process safety
from concurrent.futures import ProcessPoolExecutor, as_completed
import random


# === Paths to data ===
X_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_X.npy")
y_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_y.npy")

# === Load Data ===
X = np.load(X_path)   # (5, 8, 230000)
y = np.load(y_path)   # (5, 51, 230000)

# === Extract windows by session ===
def extract_windows_by_session(X, y, window_size=500, stride=250):
    X_sessions, y_sessions = [], []
    sessions = X.shape[0]

    for sess in range(sessions):
        X_windows, y_targets = [], []
        for start in range(0, X.shape[2] - window_size + 1, stride):
            end = start + window_size
            x_window = X[sess, :, start:end]
            y_target = y[sess, :, end-1]
            X_windows.append(x_window)
            y_targets.append(y_target)
        X_sessions.append(np.stack(X_windows))
        y_sessions.append(np.stack(y_targets))

    return X_sessions, y_sessions

X_sessions, y_sessions = extract_windows_by_session(X, y)

# === Keep only first 4 sessions for CV ===
X_sessions_cv = X_sessions[:4]
y_sessions_cv = y_sessions[:4]

class EMGDataset(Dataset):
    def __init__(self, X, y, standardize=True):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

        if standardize:
            self.mean = self.X.mean(dim=(0, 2), keepdim=True)
            self.std = self.X.std(dim=(0, 2), keepdim=True)
            self.X = (self.X - self.mean) / (self.std + 1e-8)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]



In [8]:
conv_templates = {
    "shallow_narrow": [[(16, 5, 1)], [(32, 5, 1), (32, 3, 1)]],
    "moderate": [[(16, 5, 1), (32, 3, 1)]],
    "expanding_light": [[(16, 5, 1), (32, 3, 1), (64, 3, 1)]],
    "bottleneck_light": [[(64, 5, 1), (32, 3, 1), (16, 3, 1)]],
    "oscillating_light": [[(32, 5, 1), (64, 3, 1), (32, 3, 1)]],
}

sampler = ArchitectureSampler(limit_per_shape=3, seed=42, conv_templates=conv_templates)
sampler.generate_configs()
sampler.shuffle_configs(seed=42)

configs = sampler.get_all_configs()
print("All configs (with metadata):", len(sampler.get_all_configs()))
print("Clean model configs:", len(sampler.get_model_configs()))

# For parallel training:
clean_configs = sampler.get_model_configs()


All configs (with metadata): 864
Clean model configs: 864


In [111]:
# Model and training configs

training_config = {
    "lr": 2e-3,
    "epochs": 100,
    "batch_size": 256,
    "log_every": 1
}

for config in clean_configs:
# Cross-validation runner
    cross_validator = CrossValidationManager(
    model_class=EMGConvNet,
    model_config=config,
    data=X_sessions_cv,
    labels=y_sessions_cv,
    training_config=training_config,
    dataset_class=EMGDataset,
    dataset_config={"standardize": True},
    n_folds=4
)
    # Run and save
    experiment_log = cross_validator.run()
    save_experiment_log(experiment_log, path="logs/neuralnetwork_log.json")


===== Fold 1/4 =====
Epoch   1 | Train MSE: 327.0008 | Val MSE: 308.9979 | Val RMSE: 17.5668
Epoch   2 | Train MSE: 187.2818 | Val MSE: 189.4766 | Val RMSE: 13.7503
Epoch   3 | Train MSE: 154.7449 | Val MSE: 172.5621 | Val RMSE: 13.1257
Epoch   4 | Train MSE: 138.3576 | Val MSE: 184.4517 | Val RMSE: 13.5703
Epoch   5 | Train MSE: 102.6844 | Val MSE: 145.1044 | Val RMSE: 12.0282
Epoch   6 | Train MSE: 76.7019 | Val MSE: 71.4161 | Val RMSE: 8.4385
Epoch   7 | Train MSE: 64.4216 | Val MSE: 81.0476 | Val RMSE: 8.9738
Epoch   8 | Train MSE: 61.7917 | Val MSE: 65.8257 | Val RMSE: 8.0990
Epoch   9 | Train MSE: 55.6343 | Val MSE: 63.9347 | Val RMSE: 7.9849
Epoch  10 | Train MSE: 53.8505 | Val MSE: 64.7353 | Val RMSE: 8.0325
Epoch  11 | Train MSE: 50.5719 | Val MSE: 61.0009 | Val RMSE: 7.7991
Epoch  12 | Train MSE: 48.1453 | Val MSE: 61.0059 | Val RMSE: 7.7999
Epoch  13 | Train MSE: 44.2628 | Val MSE: 54.5870 | Val RMSE: 7.3762
Epoch  14 | Train MSE: 39.9701 | Val MSE: 53.8606 | Val RMSE: 7.32

In [106]:
clean_configs[0]

{'conv_layers_config': [(128, 5, 1), (128, 3, 1)],
 'fc_layers_config': [1024, 128],
 'conv_dropouts': [0.05, 0.05],
 'fc_dropouts': [0.05, 0.05],
 'output_dim': 51,
 'verbose': False}

In [14]:
from utils import ExperimentSelector

selector = ExperimentSelector(r"D:\Uni\F422\pose-estimation-from-emg-signal-team-1\logs\neuralnetwork_log.json")

# Get top 20 by final RMSE, but filter for low variance
top_stable = selector.select(sort_by="final_avg", top_n=20, max_variance=0.15, return_full=True)

# Get top 20 by convergence potential
top_converging = selector.select(sort_by="potential", top_n=20)

# Get architectures that reached the lowest point anywhere
top_by_lowest = selector.select(sort_by="lowest_point", top_n=20)

# Get low-bias models (slow starters, strong finishers)
top_bias = selector.select(sort_by="bias", top_n=20)


In [48]:
from utils import save_experiment_log

save_experiment_log(top_stable, r"logs\stable_results.json")

In [51]:
models_summary = selector.select(sort_by="final_avg", top_n=20, return_full=True)
save_experiment_log(models_summary, r"logs\stable_results.json")

In [55]:
models_summary = selector.select(sort_by="final_avg", top_n=20)


In [57]:
# models_summary

save_experiment_log(models_summary, r"logs\summarised_stable_results.json")

In [2]:
import uuid
import os
import traceback
from models import EMGConvNet  # delay import for process safety
from concurrent.futures import ProcessPoolExecutor, as_completed
from utils import ExperimentSelector
from utils import save_experiment_log

import random


# === Paths to data ===
X_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\freemoves\freemoves_dataset_X.npy")
y_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\freemoves\freemoves_dataset_y.npy")

# === Load Data ===
X = np.load(X_path)   # (5, 8, 230000)
y = np.load(y_path)   # (5, 51, 230000)

# === Extract windows by session ===
def extract_windows_by_session(X, y, window_size=500, stride=250):
    X_sessions, y_sessions = [], []
    sessions = X.shape[0]

    for sess in range(sessions):
        X_windows, y_targets = [], []
        for start in range(0, X.shape[2] - window_size + 1, stride):
            end = start + window_size
            x_window = X[sess, :, start:end]
            y_target = y[sess, :, end-1]
            X_windows.append(x_window)
            y_targets.append(y_target)
        X_sessions.append(np.stack(X_windows))
        y_sessions.append(np.stack(y_targets))

    return X_sessions, y_sessions

X_sessions, y_sessions = extract_windows_by_session(X, y)

# === Keep only first 4 sessions for CV ===
X_sessions_cv = X_sessions[:4]
y_sessions_cv = y_sessions[:4]

class EMGDataset(Dataset):
    def __init__(self, X, y, standardize=True):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

        if standardize:
            self.mean = self.X.mean(dim=(0, 2), keepdim=True)
            self.std = self.X.std(dim=(0, 2), keepdim=True)
            self.X = (self.X - self.mean) / (self.std + 1e-8)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]



In [5]:


conv_templates = {
    "shallow_narrow": [[(16, 5, 1)], [(32, 5, 1), (32, 3, 1)]],
    "moderate": [[(16, 5, 1), (32, 3, 1)]],
    "expanding_light": [[(16, 5, 1), (32, 3, 1), (64, 3, 1)]],
    "bottleneck_light": [[(64, 5, 1), (32, 3, 1), (16, 3, 1)]],
    "oscillating_light": [[(32, 5, 1), (64, 3, 1), (32, 3, 1)]],
}

sampler = ArchitectureSampler(limit_per_shape=3, seed=42, conv_templates=conv_templates)
sampler.generate_configs()
sampler.shuffle_configs(seed=42)

configs = sampler.get_all_configs()
print("All configs (with metadata):", len(sampler.get_all_configs()))
print("Clean model configs:", len(sampler.get_model_configs()))

# For parallel training:
clean_configs = sampler.get_model_configs()

All configs (with metadata): 864
Clean model configs: 864


In [6]:
clean_configs

[{'conv_layers_config': [(32, 5, 1), (64, 3, 1), (32, 3, 1)],
  'fc_layers_config': [128, 768, 1024],
  'conv_dropouts': [0.0, 0.0, 0.0],
  'fc_dropouts': [0.0, 0.0, 0.0],
  'output_dim': 51,
  'verbose': False},
 {'conv_layers_config': [(16, 5, 1)],
  'fc_layers_config': [128, 256, 512, 1024],
  'conv_dropouts': [0.0],
  'fc_dropouts': [0.0, 0.0, 0.0, 0.0],
  'output_dim': 51,
  'verbose': False},
 {'conv_layers_config': [(16, 5, 1), (32, 3, 1)],
  'fc_layers_config': [1536, 1024, 512],
  'conv_dropouts': [0.0, 0.0],
  'fc_dropouts': [0.0, 0.0, 0.0],
  'output_dim': 51,
  'verbose': False},
 {'conv_layers_config': [(64, 5, 1), (32, 3, 1), (16, 3, 1)],
  'fc_layers_config': [128, 128, 128, 128],
  'conv_dropouts': [0.0, 0.0, 0.0],
  'fc_dropouts': [0.0, 0.0, 0.0, 0.0],
  'output_dim': 51,
  'verbose': False},
 {'conv_layers_config': [(16, 5, 1), (32, 3, 1), (64, 3, 1)],
  'fc_layers_config': [1024, 1536, 512, 1536, 1024],
  'conv_dropouts': [0.0, 0.0, 0.0],
  'fc_dropouts': [0.0, 0.0, 

In [11]:
# Model and training configs

training_config = {
    "lr": 2e-3,
    "epochs": 100,
    "batch_size": 256,
    "log_every": 1
}

for config in clean_configs:
# Cross-validation runner
    cross_validator = CrossValidationManager(
    model_class=EMGConvNet,
    model_config=config,
    data=X_sessions_cv,
    labels=y_sessions_cv,
    training_config=training_config,
    dataset_class=EMGDataset,
    dataset_config={"standardize": True},
    n_folds=4
)
    # Run and save
    experiment_log = cross_validator.run()
    save_experiment_log(experiment_log, path="logs/freemoves_neuralnetwork_log.json")


===== Fold 1/4 =====
Epoch   1 | Train MSE: 319.2673 | Val MSE: 307.4244 | Val RMSE: 17.4873
Epoch   2 | Train MSE: 176.0164 | Val MSE: 135.5665 | Val RMSE: 11.6229
Epoch   3 | Train MSE: 139.2678 | Val MSE: 135.4774 | Val RMSE: 11.6244
Epoch   4 | Train MSE: 129.6738 | Val MSE: 139.2633 | Val RMSE: 11.7905
Epoch   5 | Train MSE: 119.0135 | Val MSE: 143.3925 | Val RMSE: 11.9583
Epoch   6 | Train MSE: 103.5642 | Val MSE: 123.8360 | Val RMSE: 11.0805
Epoch   7 | Train MSE: 94.8025 | Val MSE: 115.9948 | Val RMSE: 10.7558
Epoch   8 | Train MSE: 96.3675 | Val MSE: 118.8454 | Val RMSE: 10.8930
Epoch   9 | Train MSE: 80.7676 | Val MSE: 129.8621 | Val RMSE: 11.3915
Epoch  10 | Train MSE: 77.5662 | Val MSE: 126.4076 | Val RMSE: 11.2360
Epoch  11 | Train MSE: 70.7823 | Val MSE: 121.7867 | Val RMSE: 11.0302
Epoch  12 | Train MSE: 78.6520 | Val MSE: 139.8311 | Val RMSE: 11.8193
Epoch  13 | Train MSE: 70.5814 | Val MSE: 212.2184 | Val RMSE: 14.5333
Epoch  14 | Train MSE: 79.7238 | Val MSE: 134.221

KeyboardInterrupt: 

In [15]:
clean_configs[549]

{'conv_layers_config': [(16, 5, 1)],
 'fc_layers_config': [128, 256, 512, 768, 1536],
 'conv_dropouts': [0.05],
 'fc_dropouts': [0.05, 0.05, 0.05, 0.05, 0.05],
 'output_dim': 51,
 'verbose': False}

In [8]:
training_config = {
    "lr": 2e-3,
    "epochs": 100,
    "batch_size": 256,
    "log_every": 1
}


valid = CrossValidationManager(
    model_class=EMGConvNet,
    model_config=clean_configs[0],
    data=X_sessions_cv,
    labels=y_sessions_cv,
    training_config=training_config,
    dataset_class=EMGDataset,
    dataset_config={"standardize": True},
    n_folds=4
)

result = valid.run()


===== Fold 1/4 =====
Epoch   1 | Train MSE: 329.5155 | Val MSE: 231.1083 | Val RMSE: 15.1608
Epoch   2 | Train MSE: 162.7434 | Val MSE: 133.1607 | Val RMSE: 11.5276
Epoch   3 | Train MSE: 149.1116 | Val MSE: 135.8898 | Val RMSE: 11.6462
Epoch   4 | Train MSE: 131.2083 | Val MSE: 128.5051 | Val RMSE: 11.3301
Epoch   5 | Train MSE: 115.3413 | Val MSE: 126.8589 | Val RMSE: 11.2331
Epoch   6 | Train MSE: 105.8394 | Val MSE: 127.0806 | Val RMSE: 11.2550
Epoch   7 | Train MSE: 95.1356 | Val MSE: 129.0330 | Val RMSE: 11.3499
Epoch   8 | Train MSE: 87.6620 | Val MSE: 132.6381 | Val RMSE: 11.5135
Epoch   9 | Train MSE: 91.1564 | Val MSE: 135.9686 | Val RMSE: 11.6571
Epoch  10 | Train MSE: 92.4268 | Val MSE: 139.2912 | Val RMSE: 11.7472
Epoch  11 | Train MSE: 80.3853 | Val MSE: 130.4522 | Val RMSE: 11.4165
Epoch  12 | Train MSE: 71.6207 | Val MSE: 140.5131 | Val RMSE: 11.8454
Epoch  13 | Train MSE: 65.8187 | Val MSE: 128.7005 | Val RMSE: 11.3122
Epoch  14 | Train MSE: 80.8175 | Val MSE: 120.389

In [9]:
result

{'architecture': {'conv_layers_config': [(32, 5, 1), (64, 3, 1), (32, 3, 1)],
  'fc_layers_config': [128, 768, 1024],
  'conv_dropouts': [0.0, 0.0, 0.0],
  'fc_dropouts': [0.0, 0.0, 0.0],
  'output_dim': 51,
  'verbose': False},
 'training_config': {'lr': 0.002,
  'epochs': 100,
  'batch_size': 256,
  'log_every': 1},
 'folds': [{'fold_number': 0,
   'metrics': {'train_losses': [329.5155011996981,
     162.7434360997928,
     149.11155196299774,
     131.2082595801626,
     115.34129098745493,
     105.83941525944523,
     95.13555814868549,
     87.66201568216978,
     91.15637604881667,
     92.42677359688528,
     80.3853486597004,
     71.62065394380633,
     65.81868206751284,
     80.81753453068207,
     77.28129756763683,
     61.51629670844021,
     54.12058568700508,
     55.46285274318239,
     54.91608318179192,
     48.271369634751444,
     45.8388532674493,
     50.08446706913856,
     46.3306885048457,
     39.27076467113949,
     44.81207348926958,
     44.28826769951652

In [12]:
# models_summary

selector = ExperimentSelector(r"D:\Uni\F422\pose-estimation-from-emg-signal-team-1\logs\freemoves_neuralnetwork_log.json")

# Get top 20 by final RMSE, but filter for low variance
top_stable = selector.select(sort_by="final_avg", top_n=20, max_variance=0.15, return_full=True)

# Get top 20 by convergence potential
top_converging = selector.select(sort_by="potential", top_n=20)

# Get architectures that reached the lowest point anywhere
top_by_lowest = selector.select(sort_by="lowest_point", top_n=20)

# Get low-bias models (slow starters, strong finishers)
top_bias = selector.select(sort_by="bias", top_n=20)

models_summary = selector.select(sort_by="final_avg", top_n=20)
save_experiment_log(models_summary, r"logs\freemoves_summarised_stable_results.json")